In [1]:
import numpy as np

In [145]:
import numpy as np
import time


def sigmoid_activation(x):
    return 1. / (1. + np.exp(-x))


class Controller:
    def control(self):
        left = np.random.choice([1, 0])
        right = np.random.choice([1, 0])
        front = np.random.choice([1, 0])
        return [left, right, front]


class robot_controller(Controller):
    def __init__(self, _n_hidden):
        self.n_hidden = [_n_hidden]

    def set(self, controller, n_inputs):
        if self.n_hidden[0] > 0:
            # Biases for the n hidden neurons
            self.bias1 = controller[:self.n_hidden[0]].reshape(1, self.n_hidden[0])
            # Weights for the connections from the inputs to the hidden nodes
            weights1_slice = n_inputs * self.n_hidden[0] + self.n_hidden[0]
            self.weights1 = controller[self.n_hidden[0]:weights1_slice].reshape((n_inputs, self.n_hidden[0]))

            # Biases and weights for the second layer
            self.bias2 = controller[weights1_slice:weights1_slice + 3].reshape(1, 3)
            self.weights2 = controller[weights1_slice + 3:].reshape((self.n_hidden[0], 3))

    def control(self, inputs, controller):
        # Normalizes the input using min-max scaling
        input_min = min(inputs)
        input_max = max(inputs)
        if input_max - input_min != 0:
            inputs = (inputs - input_min) / float(input_max - input_min)
        else:
            inputs = inputs * 0

        print(f"Normalized inputs: {inputs}")

        if self.n_hidden[0] > 0:
            output1 = sigmoid_activation(inputs.dot(self.weights1) + self.bias1)
            print(f"Hidden layer output: {output1}")
            output = sigmoid_activation(output1.dot(self.weights2) + self.bias2)[0]
        else:
            bias = controller[:3].reshape(1, 3)
            weights = controller[3:].reshape((len(inputs), 3))
            output = sigmoid_activation(inputs.dot(weights) + bias)[0]

        print(f"Output layer: {output}")

        # Determine actions based on output
        if output[0] > 0.5:
            action = 'move_forward'
        elif output[1] > 0.5:
            action = 'turn_left'
        elif output[2] > 0.5:
            action = 'turn_right'
        else:
            action = 'move_forward'  # Default action

        return action


def move_forward(rob, speed, duration):
    rob.move_blocking(left_speed=speed, right_speed=speed, millis=duration)
    time.sleep(duration / 1000)


def turn_left(rob, speed, duration):
    rob.move_blocking(left_speed=-speed, right_speed=speed, millis=duration)
    time.sleep(duration / 1000)


def turn_right(rob, speed, duration):
    rob.move_blocking(left_speed=speed, right_speed=-speed, millis=duration)
    time.sleep(duration / 1000)


# Initialize the controller with a certain number of hidden neurons
n_hidden = 1
controller = robot_controller(n_hidden)

# Improved controller weights initialization for better diversity
controller_weights = np.random.uniform(-10, 10, 5 * n_hidden + 3 + n_hidden * 3 + n_hidden)

controller.set(controller_weights, n_inputs=5)  # 5 sensor inputs


# Example sensor input
sensor_inputs = np.random.uniform(low=0.0, high=10.0, size=5)

# Get action from controller
action = controller.control(sensor_inputs, controller_weights)
print(f"Action: {action}")

# for i in range(10):
#     # Example sensor input
#     sensor_inputs = np.random.uniform(low=0.0, high=10.0, size=5)

#     # Get action from controller
#     action = controller.control(sensor_inputs, controller_weights)
#     print(f"Action: {action}")


Normalized inputs: [0.10150955 1.         0.         0.32038778 0.49398084]
Hidden layer output: [[0.99993102]]
Output layer: [2.84494475e-03 5.79585730e-01 6.38790374e-05]
Action: turn_left
Normalized inputs: [0.46866872 1.         0.         0.01737162 0.61320479]
Hidden layer output: [[0.99469599]]
Output layer: [2.82167618e-03 5.87998981e-01 6.46783793e-05]
Action: turn_left
Normalized inputs: [0.59704176 0.58879143 0.61213656 1.         0.        ]
Hidden layer output: [[0.99979553]]
Output layer: [2.8443401e-03 5.7980409e-01 6.3899601e-05]
Action: turn_left
Normalized inputs: [0.53324933 0.         1.         0.04189066 0.9628487 ]
Hidden layer output: [[0.00299855]]
Output layer: [5.94163863e-04 9.99008341e-01 6.81788201e-04]
Action: turn_left
Normalized inputs: [0.81720468 1.         0.49661727 0.         0.35010834]
Hidden layer output: [[0.50529153]]
Output layer: [1.30852998e-03 9.73215824e-01 2.06836850e-04]
Action: turn_left
Normalized inputs: [0.79714823 0.         0.7420

In [134]:
controller_weights = np.random.uniform(-10, 10, 5 * n_hidden + 3 + n_hidden * 3 + n_hidden)
controller_weights

array([ 0.91181807, -4.24424786,  2.78504491,  7.5579936 ,  5.11414774,
       -0.54370841,  4.25691983,  5.58766084, -4.89532451, -6.11459226,
       -5.20273361, -6.59258421])